# 📦 Olist E-Commerce Logistics & Delivery Analysis
> **End-to-End Data Engineering & Business Intelligence Project**

## 📝 Project Overview
This project analyzes a real-world dataset from **Olist**, the largest department store in Brazilian marketplaces. The goal is to identify logistics bottlenecks, measure delivery performance against estimates, and audit seller reliability to improve customer satisfaction.

## 🛠️ Tech Stack
* **Python (Pandas/NumPy):** Data Cleaning, ETL, and Feature Engineering.
* **SQL (SQLite):** Relational data storage and business logic queries.
* **Power BI:** Geospatial visualisation and executive dashboarding.

## 🎯 Project Objectives
1.  **Data Engineering:** Clean 100k+ records and merge 3+ relational tables.
2.  **KPI Creation:** Engineer `actual_wait_time` and `is_late` metrics.
3.  **Logistics Insight:** Identify regional delivery delays across Brazil's 27 states.
4.  **Seller Audit:** Flag high-risk sellers with >15% late-delivery rates.

---
## 🚀 Phase 1: Data Ingestion & Cleaning
*Starting by importing raw datasets and handling missing values...*

In [1]:
import pandas as pd

In [2]:
# Exact column names from the Olist dataset
date_cols = [
    'order_purchase_timestamp', 
    'order_delivered_customer_date', 
    'order_estimated_delivery_date'
]

# Load the data
orders = pd.read_csv('../data/olist_orders_dataset.csv', parse_dates=date_cols)
items = pd.read_csv('../data/olist_order_items_dataset.csv')
payments = pd.read_csv('../data/olist_order_payments_dataset.csv')

# Verify it worked
print("--- Check Column Types ---")
print(orders.dtypes[date_cols])

print("\n--- Check for Missing Values ---")
print(orders.isnull().sum())

--- Check Column Types ---
order_purchase_timestamp         datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

--- Check for Missing Values ---
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64


In [3]:
# Create a 'clean' version of orders by removing rows where delivery date is missing
# .copy() ensures we don't accidentally change the original 'orders' dataframe
orders_clean = orders.dropna(subset=['order_delivered_customer_date']).copy()

#Summary of what happed?
print(f"Total Original Orders: {len(orders)}")
print(f"Orders after removing nulls: {len(orders_clean)}")
print(f"Removed Rows: {len(orders) - len(orders_clean)}")

Total Original Orders: 99441
Orders after removing nulls: 96476
Removed Rows: 2965


**Rows with missing delivery dates were removed because delivery delay cannot be calculated without both order and delivery timestamps.

In [4]:
#Actual time it took for a customer to get the order
orders_clean['actual_wait_time']=(orders_clean['order_delivered_customer_date'] - orders_clean['order_purchase_timestamp']).dt.days

# 2. Calculate if the order was late compared to the estimate
orders_clean['delay_days']= (orders_clean['order_delivered_customer_date'] - orders_clean['order_estimated_delivery_date']).dt.days
orders_clean['is_late']= orders_clean['delay_days'] > 0

#Outcome
print(orders_clean[['actual_wait_time', 'delay_days','is_late']])

       actual_wait_time  delay_days  is_late
0                     8          -8    False
1                    13          -6    False
2                     9         -18    False
3                    13         -13    False
4                     2         -10    False
...                 ...         ...      ...
99436                 8         -11    False
99437                22          -2    False
99438                24          -6    False
99439                17         -21    False
99440                 7         -18    False

[96476 rows x 3 columns]


In [5]:
#Merging orders with items(This merge product_id to seller_id)
sellers = pd.read_csv('../data/olist_sellers_dataset.csv')
df_merged = orders_clean.merge(items, on='order_id', how = 'inner')
df_final = df_merged.merge(sellers, on='seller_id', how ='inner')

print(f"New dataset has {df_final.shape[0]} rows and {df_final.shape[1]} columns.")
print(df_final.columns)

New dataset has 110196 rows and 20 columns.
Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'actual_wait_time', 'delay_days', 'is_late', 'order_item_id',
       'product_id', 'seller_id', 'shipping_limit_date', 'price',
       'freight_value', 'seller_zip_code_prefix', 'seller_city',
       'seller_state'],
      dtype='object')


In [6]:
# Data Export & System Integration
import sqlite3

# 1. Create a database file for your SQL work
conn = sqlite3.connect('../data/olist_analysis.db')
df_final.to_sql('delivery_performance', conn, if_exists='replace', index=False)
conn.close()

# 2. Save a clean CSV for Power BI
df_final.to_csv('../data/olist_for_powerbi.csv', index=False)

print("Files created! Act 1 (Python) is finished.")

Files created! Act 1 (Python) is finished.


In [7]:
# --- EXPORT FOR POWER BI ---
import os
# Create the 'data' folder if it doesn't exist
if not os.path.exists('../data'):
    os.makedirs('../data')

# Export the final merged dataframe to CSV
df_final.to_csv('../data/olist_for_powerbi.csv', index=False)

print("✅ Success! 'olist_for_powerbi.csv' is now in your data folder.")

✅ Success! 'olist_for_powerbi.csv' is now in your data folder.


In [8]:
import sqlite3
import pandas as pd

# 1. Connect to the database we created
conn = sqlite3.connect('../data/olist_analysis.db')

# 2. Check if the table 'delivery_performance' exists
table_check = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("--- Tables in Database ---")
print(table_check)

# 3. Run a test query to see the first 5 rows
test_query = "SELECT * FROM delivery_performance LIMIT 5;"
df_test = pd.read_sql(test_query, conn)

print("\n--- Data Preview ---")
display(df_test)

conn.close()

--- Tables in Database ---
                   name
0  delivery_performance

--- Data Preview ---


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,actual_wait_time,delay_days,is_late,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_zip_code_prefix,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,8,-8,0,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,9350,maua,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,13,-6,0,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,31570,belo horizonte,SP
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,9,-18,0,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,14840,guariba,SP
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,13,-13,0,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,31842,belo horizonte,MG
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,2,-10,0,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,8752,mogi das cruzes,SP


## 🏁 Project Phase 1: Complete
The data pipeline is now fully operational.
* **Cleaning:** Handled missing values and date formatting.
* **Engineering:** Created `actual_wait_time` and `is_late` KPIs.
* **Storage:** Data is persisted in `olist_analysis.db` and `olist_for_powerbi.csv`.

**Next Step:** Proceeding to **Power BI** for executive dashboarding.